# Paquetes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

# Parametros

In [3]:
# MODEL

_LANGUAGE_         = 'es'
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig()
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'parallel_adapter_checkPoints'
_LOGGING_STEPS_    = 50


# TRAIN

_NO_GPUS_          = 5
_BATCH_SIZE_       = 100
_EPOCHS_           = 20

# Dataset

In [4]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'gran britain' : 2,
                  'ireland'      : 3,
                  'new zeland'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [5]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [6]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer

if _LANGUAGE_ == 'es':
    tokenizer = AutoTokenizer.from_pretrained('pysentimiento/robertuito-base-cased')
    
vocab = tokenizer.get_vocab()

In [7]:
# CREATE ONE INSTANCE PER DATA SPLIT

from DatasetPAN17 import BasePAN17, DatasetPAN17

baseTrain = BasePAN17(Dir              = 'data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_)

baseTest  = BasePAN17(Dir              = 'data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done

Total Instances: 56000



In [8]:
# CREATE DATA LOADER FOR EVERY TASK

tasks = ['gender', 'variety']

dataset_dict = {}
for task in tasks:
    dataset_dict[task] = DatasetPAN17(Base_Dataset = baseTrain, label = task)

# Model

In [9]:
from transformers import RobertaConfig, RobertaModelWithHeads

if _LANGUAGE_ == 'es':
    
    config = RobertaConfig.from_pretrained(
        "pysentimiento/robertuito-base-cased",
        num_labels=2,
    )
    
    model = RobertaModelWithHeads.from_pretrained(
        "pysentimiento/robertuito-base-cased",
        config=config,
    )

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/adapters/models/roberta.py:250: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/adapters/models/roberta.py:228: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaModelWithHeads: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (

# Adapters

In [10]:
num_v           = len(baseTrain.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

# Add adapters
for task_name in tasks:
    
    model.add_adapter(
        adapter_name = task_name, 
        config       = _ADAPTER_CONFIG_
    )
    
    model.add_classification_head(
        head_name    = task_name,
        num_labels   = num_labels_dict[task_name],
      )

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

RobertaModelWithHeads(
  (shared_parameters): ModuleDict()
  (roberta): RobertaModel(
    (shared_parameters): ModuleDict()
    (invertible_adapters): ModuleDict()
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (prefix_tuning): PrefixTuningShim(


# Training

In [11]:
from Training import train_model_with_adapters

train_model_with_adapters(model         = model,
                          dataset_dict  = dataset_dict,
                          epochs        = _EPOCHS_,
                          batch_size    = _BATCH_SIZE_,
                          no_gpus       = _NO_GPUS_,
                          output_dir    = _OUTPUT_DIR_,
                          logging_steps = _LOGGING_STEPS_)

/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 84000
  Num Epochs = 20
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 500
  Gradient Accumulation steps = 1
  Total optimization steps = 3360
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
50,0.659100
100,0.576900
150,0.543800
200,0.532100
250,0.510200
300,0.514500
350,0.501600
400,0.471900
450,0.469200
500,0.468700


Saving model checkpoint to parallel_adapter_checkPoints/gender/checkpoint-500
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-500/gender/head_config.json
Module w

Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-3000/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-3000/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-3000/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-3000/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-3000/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-3000/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-3000/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-3000/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/gender/checkpoint-3000/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints/gender/checkpoint-3000/g

Step,Training Loss
50,1.107800
100,0.775000
150,0.728500
200,0.686500
250,0.668500
300,0.659600
350,0.605000
400,0.583600
450,0.583500
500,0.568000


Saving model checkpoint to parallel_adapter_checkPoints/variety/checkpoint-500
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-500/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-500/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-500/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-500/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-500/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-500/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-500/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-500/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-500/gender/head_config.jso

Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-3000/gender/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-3000/gender/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-3000/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-3000/gender/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-3000/variety/adapter_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-3000/variety/pytorch_adapter.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-3000/variety/head_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpoint-3000/variety/pytorch_model_head.bin
Configuration saved in parallel_adapter_checkPoints/variety/checkpoint-3000/gender/head_config.json
Module weights saved in parallel_adapter_checkPoints/variety/checkpo

# Testing

In [12]:
from TestingPAN17 import test_model_with_adapters

accuracy = test_model_with_adapters(model, baseTest)

100%|███████████████████████████████████████| 2800/2800 [24:08<00:00,  1.93it/s]
